In [2]:
from langfuse import Langfuse
from anthropic import AnthropicBedrock
from compiler.core import Compiler
from application import Application

In [3]:
langfuse_client = Langfuse()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
m_claude = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [6]:
builder = Application(m_claude, compiler)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [7]:
prepare_res = builder.prepare_bot(["bot that greets me by name"], bot_id="1337_420", capabilities=[])

Processing transition: [] FsmState.TYPESPEC
Processing transition: [<FsmState.TYPESPEC: 'typespec'>] FsmState.WAIT
{'description': '', 'user_requests': ['bot that greets me by name'], 'typespec_schema': <fsm_core.typespec.Success object at 0x7f28d815e600>}


In [8]:
prepare_res

ApplicationPrepareOut(refined_description=RefineOut(refined_description='', error_output=None), capabilities=CapabilitiesOut(capabilities=[], error_output=None), typespec=TypespecOut(reasoning="Based on the user request, I need to design a simple greeter bot that can:\n1. Store and retrieve user names\n2. Generate personalized greetings for users\n3. Update a user's name if they want to change it\n\nFrom the request, I infer the primary function is greeting by name, which implies at minimum we need:\n- A function to set/update the user's name\n- A function to generate a greeting with the stored name\n\nFor the models:\n- We need a model to represent a user (with at minimum a name property)\n- We need a model for the greeting request (which might include time of day or other context)\n- We need a model for updating user information\n\nThe interface should be focused on greeting functionality with appropriate scenarios.", typespec_definitions='model User {\n  name: string;\n}\n\nmodel Gr

In [9]:
prepare_res.typespec.typespec_definitions

'model User {\n  name: string;\n}\n\nmodel GreetingRequest {\n  timeOfDay?: string;\n  includeFormalTitle?: boolean;\n}\n\nmodel UpdateUserRequest {\n  currentName: string;\n  newName: string;\n}\n\ninterface GreeterBot {\n  @scenario(\n  """\n  Scenario: Greeting a user by name\n  When user says "Hello, my name is John"\n  Then system should extract:\n    - Name: "John"\n  And respond with a personalized greeting\n  Examples:\n    | Input                       | Expected Response             |\n    | "Hi, I\'m Sarah"             | "Hello, Sarah! Nice to meet you!" |\n    | "Call me Dr. Smith"         | "Hello, Dr. Smith! How are you today?" |\n  """)\n  @llm_func("Store a user\'s name and generate a personalized greeting")\n  setUserName(options: User): string;\n\n  @scenario(\n  """\n  Scenario: Generating greeting with context\n  When user asks "Greet me"\n  Then system should use stored name and context to create greeting\n  Examples:\n    | Stored Name | Time of Day | Expected Res

In [10]:
reasoning = prepare_res.typespec.reasoning
typespec_schema = prepare_res.typespec.typespec_definitions
fake_msg = f"<reasoning>{reasoning}</reasoning><typespec>{typespec_schema}</typespec>"

In [11]:
final_res = builder.update_bot(
    typespec_schema=fake_msg,
    bot_id="1337_420",
    capabilities=[],
)

Typespec schema: <reasoning>Based on the user request, I need to design a simple greeter bot that can:
1. Store and retrieve user names
2. Generate personalized greetings for users
3. Update a user's name if they want to change it

From the request, I infer the primary function is greeting by name, which implies at minimum we need:
- A function to set/update the user's name
- A function to generate a greeting with the stored name

For the models:
- We need a model to represent a user (with at minimum a name property)
- We need a model for the greeting request (which might include time of day or other context)
- We need a model for updating user information

The interface should be focused on greeting functionality with appropriate scenarios.</reasoning><typespec>model User {
  name: string;
}

model GreetingRequest {
  timeOfDay?: string;
  includeFormalTitle?: boolean;
}

model UpdateUserRequest {
  currentName: string;
  newName: string;
}

interface GreeterBot {
  @scenario(
  """
 

In [9]:
final_res

ApplicationOut(refined_description=RefineOut(refined_description='', error_output=None), capabilities=CapabilitiesOut(capabilities=[], error_output=None), typespec=TypespecOut(reasoning="Based on the conversation, I need to create a simple greeting bot that greets users by their name. The bot should be able to:\n1. Greet a user by name\n2. Remember a user's name for future interactions\n3. Update a user's name if they want to change it\n\nThe main models needed would be:\n- A model for storing user information (name)\n- A model for greeting options/parameters\n\nThe interface should have functions for:\n- Setting/saving a user's name\n- Retrieving a greeting with the user's name\n- Updating a user's name if needed\n\nEach function will need scenarios to illustrate usage.", typespec_definitions='model UserInfo {\n    name: string;\n}\n\nmodel GreetingOptions {\n    timeOfDay: string;\n    isFirstTime: boolean;\n}\n\nmodel UpdateNameOptions {\n    currentName: string;\n    newName: strin

In [ ]:
from core.interpolator import Interpolator

import os

# Create the botbuild directory if it doesn't exist
if not os.path.exists("../botbuild"):
    os.makedirs("../botbuild")

interpolator = Interpolator(".")

interpolator.bake(final_res, "../../bot_result")


FileNotFoundError: [Errno 2] No such file or directory: '../botbuild/templates'